Some label masks have one class missing. So need to check and figure out how to fix it.
Need the shape of `target` and `predicted` mask to be same for loss calcualtion.

In [1]:
# Import modules

import os, glob
import numpy as np
import torchio as tio

print('TorchIO version:', tio.__version__)

from platform import python_version
print(python_version())

TorchIO version: 0.18.78
3.10.4


In [2]:
dataset_dir = '/media/trojan/evo/3D-CT-Artery-Segmentation/data/data_new/data_original'

In [3]:
images_dir = os.path.join(dataset_dir, 'train/volumes')
labels_dir = os.path.join(dataset_dir, 'train/masks')
image_paths = sorted(glob.glob(images_dir + '/' + '*.nii.gz'))
label_paths = sorted(glob.glob(labels_dir + '/' + '*.nii.gz'))
assert len(image_paths) == len(label_paths)

training_subjects = []
for (image_path, label_path) in zip(image_paths, label_paths):
    subject = tio.Subject(
        image=tio.ScalarImage(image_path),
        label=tio.LabelMap(label_path),
    )
    training_subjects.append(subject)
training_dataset = tio.SubjectsDataset(training_subjects)
print('Dataset size:', len(training_dataset), 'training subjects')

Dataset size: 44 training subjects


Check all data. If size is different i.e. label is missing, then print the name of that volume.

In [4]:
# Check issue masks
issue_masks = []

for label_path in label_paths:
    label = tio.LabelMap(label_path)
    unique_labels = np.unique(label.data)
    #print(unique_labels.shape[0])
    
    # Check label sizes and save the issue masks
    if unique_labels.shape[0] > 3:
        print("Size is not correct.")
        print(label_path.split("/")[-1])
        print(unique_labels)
        print('\n')
        issue_masks.append(label_path.split("/")[-1])

Size is not correct.
18.nii.gz
[0 1 2 3]


Size is not correct.
6.nii.gz
[0 1 2 3]




# Extra class fix

The `extra` label issue is because of separate label for left and right **facial artery**. I can fix it by combining them into one label.
However I need to do it before apply `transform.Compose()`.

## Apply on training data

In [5]:
out_dir = '/media/trojan/evo/3D-CT-Artery-Segmentation/data/data_new/fixed_masks'

In [7]:
for mask in issue_masks:
    path = os.path.join(labels_dir, mask)
    label = tio.LabelMap(path)
    print("Before fix: ", np.unique(label.data))
    label.data[label.data==3] = 2
    print("After fix: ", np.unique(label.data))
    label.save(os.path.join(out_dir, '{}.nii.gz'.format(mask)))

Before fix:  [0 1 2 3]
After fix:  [0 1 2]
Before fix:  [0 1 2 3]
After fix:  [0 1 2]


## Check Shapes

In [8]:
for mask in glob.glob(out_dir + "/*.nii.gz"):
    label = tio.LabelMap(mask)
    unique_labels = np.unique(label.data)
    print(unique_labels)

[0 1 2]
[0 1 2]
